In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
a= open ('dump.json', "r")
data=json.load(a)
df = pd.read_json("dump.json")

In [3]:
book = df["book"].iloc[1]
dfmuslim = df.loc[df["book"]==book]

In [4]:
pd.set_option('display.max_rows',20)
for col in dfmuslim.columns:
    #print(col,":",dfmuslim.loc[:,col].nunique())
    print(dfmuslim.loc[:,col].value_counts())

{'$oid': '626f317fe7116a8530050592'}    1
{'$oid': '626f31a8e7116a8530051913'}    1
{'$oid': '626f31a8e7116a853005191f'}    1
{'$oid': '626f31a8e7116a853005191e'}    1
{'$oid': '626f31a8e7116a853005191d'}    1
                                       ..
{'$oid': '626f3194e7116a8530050f50'}    1
{'$oid': '626f3194e7116a8530050f4f'}    1
{'$oid': '626f3194e7116a8530050f4e'}    1
{'$oid': '626f3194e7116a8530050f4d'}    1
{'$oid': '626f31bde7116a85300522d8'}    1
Name: _id, Length: 7495, dtype: int64
{'ar_name': 'مسلم', 'en_name': 'Muslim'}    7495
Name: author, dtype: int64
{'ar_name': 'كتاب صحيح مسلم - ت عبد الباقي'}    7495
Name: book, dtype: int64
1727    7495
Name: book_id, dtype: int64
1375    1
113     1
1960    1
955     1
5470    1
       ..
7073    1
3899    1
2271    1
891     1
4226    1
Name: page_id, Length: 7495, dtype: int64
[فهرس الكتاب, ٣٦ - كتاب الأشربة, ٦ - باب: النهي عن الانتباذ في المزفت والدباء والحنتم والنقير، و بيان أنه منسوخ، وأنه اليوم حلال، ما لم يصر مسكرا]       

In [5]:
#df_trial = pd.DataFrame()
#for i in rows:
#    df_trial = pd.join(df_trial,dfmuslim.loc[:,'clean_texts'].iloc[i])
df_trial = dfmuslim.loc[:,'clean_texts']

In [6]:
def is_numb(word):
    for w in word:
        if w  in ar_num:
            isit = True
        else:
            isit = False
            break
    return isit

def cut(words):
    sep = 0
    for w in range(0,len(words)-3):
        w1 = words[w]
        w2 = words[w+1]
        w3 = words[w+2]
        b1 = is_numb(w1)
        b2 = is_numb(w2)
        b3 = is_numb(w3)
        if b1 and b2 and b3 :
            sep = w
            break
    return words[sep:]
                    
        
def translate(num):
    try:
        eng_equi=''
        for i in range(0,len(num)):
                temp = ar_num.index(num[i])
                eng_equi += eng_num[temp]
    except:
        temp = 0
    return eng_equi

    
hadith = []  
index = []
hadith_number = []
ar_num = ['٨','٦','٩','٥','٣','٧','٢','٤','١',' ','-','(', ')','م', '٠','ب']
eng_num = ['8', '6', '9', '5', '3', '7', '2', '4','1', ' ', '-', '(', ')', 'M','0','B']

count=0
for i in (df_trial):
    j= " ".join(i)
    words = j.split()
    w1 = (is_numb(words[0]))
    w2 = (is_numb(words[1]))
    w3 = (is_numb(words[2]))
    if not w1:
        if len(words)>20:
            if (words[2]  =="بَاب") or (words[2] == "باب") or (words[2]  =="بَاب:") or (words[2] == "باب:"):
                c = cut(words)
                had = " ".join(cut(words))
                hadith.append(had)
                index.append(df_trial.index[count])
                hadith_number.append(translate(c[0]+c[1]+c[2]))
            else:
                had = " ".join(words)
                hadith.append(had)
                index.append(df_trial.index[count])
                hadith_number.append(0)
    elif not w2:
        if (words[2]  =="بَاب") or (words[2] == "باب") or (words[2]  =="بَاب:") or (words[2] == "باب:"):
            c = cut(words)
            had = " ".join(cut(words))
            hadith.append(had)
            index.append(df_trial.index[count])
            hadith_number.append(translate(c[0]+c[1]+c[2]))
        else:
            had = " ".join(words)
            hadith.append(had)
            index.append(df_trial.index[count])
            hadith_number.append(translate(words[0]))
    elif not w3:
        if (words[2]  =="بَاب") or (words[2] == "باب") or (words[2]  =="بَاب:") or (words[2] == "باب:"):
            c = cut(words)
            had = " ".join(c)
            hadith.append(had)
            index.append(df_trial.index[count])
            hadith_number.append(translate(c[0]+c[1]+c[2]))
        else:
            had = " ".join(words)
            hadith.append(had)
            index.append(df_trial.index[count])
            hadith_number.append(translate(words[0]+words[1]))
    else:
        had = " ".join(words)
        hadith.append(had)
        index.append(df_trial.index[count])
        hadith_number.append(translate(words[0]+words[1]+words[2]))
    count+=1


    #hadith.append(j)
    #index.append(dfmuslim.index[i])
    
        


In [7]:
new_df = pd.DataFrame(hadith,index = index,columns=['clean_texts'])
hadith_number = pd.DataFrame(hadith_number, columns = ['hadith_number'])

In [8]:
new_df.drop(columns=['clean_texts'], inplace = True)
new_df = new_df.join(dfmuslim,how='left')
new_df = new_df.reset_index()

In [9]:
pd.set_option('display.max_rows',7650)
new_df = pd.concat([new_df, hadith_number],axis=1)
#new_df.sort_values(by='page_id',inplace=True)

In [10]:
anomaly_index = []
indicator = []
for i in range(len(new_df)):
    if new_df.iloc[i].hadith_number == 0:
        if new_df.iloc[i].clean_texts[0][0] == '[':
            anomaly_index.append(i)
            indicator.append(0)
    elif new_df.iloc[i].hadith_number.startswith('(') and new_df.iloc[i].hadith_number.endswith(')'):
            anomaly_index.append(i)
            indicator.append(1)
anomaly_df = new_df.iloc[anomaly_index]           
new_df.drop(anomaly_index, inplace=True)

In [11]:
len(new_df)

6808

In [12]:
len(anomaly_index)

677

In [13]:
sum(indicator)

655

In [14]:
anomaly_hadith = []
anomaly_indexes = []
anomaly_hadith_number = []
for i in range(len(anomaly_df)):
    c = " ".join(anomaly_df.iloc[i].clean_texts[1:])
    if len(c)>0:
        anomaly_hadith.append(c)
        anomaly_indexes.append(anomaly_df.index[i])
anomaly_df.drop(columns=['clean_texts', 'hadith_number'],inplace=True)

newer_df = pd.DataFrame(anomaly_hadith,index = anomaly_indexes,columns=['clean_texts'])
newer_df = newer_df.join(anomaly_df,how='left')



/Users/hussami/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [15]:
anomaly_hadith_number = []
for i in anomaly_hadith:
    words = i.split()
    w1 = is_numb(words[0])
    w2 = is_numb(words[1])
    w3 = is_numb(words[2])
    if w1:
        if w2:
            if w3:
                anomaly_hadith_number.append(translate(words[0]+words[1]+words[2]))
            else:
                anomaly_hadith_number.append(translate(words[0]+words[1]))
        else:
            anomaly_hadith_number.append(translate(words[0]))
    else:
        anomaly_hadith_number.append(0)
    


In [16]:
anomaly_hadith_numberdf = pd.DataFrame(anomaly_hadith_number, index=anomaly_indexes,columns = ['hadith_number'])

In [17]:
newer_df = pd.concat([newer_df, anomaly_hadith_numberdf],axis=1)

In [18]:
new_df = pd.concat([newer_df,new_df],ignore_index=True)

In [19]:
new_df.loc[:,['clean_texts', 'shamela_link','hadith_number','page_id']].sort_values(by='page_id')

,clean_texts,shamela_link,hadith_number,page_id
4364,"[بسم الله الرحمن الرحيم, الحمد لله رب العالمين...",https://shamela.ws/book/1727/1,0,1
3908,[تحت أيديهم. وقد تضافرت على تصحيحها كل من: الع...,https://shamela.ws/book/1727/2,0,2
2593,[{هو الذي بعث في الأميين رسولا منهم يتلو عليهم...,https://shamela.ws/book/1727/3,0,3
1819,[تَؤُولُ بِهِ الْحَالُ إِنْ شَاءَ اللَّهُ، عَا...,https://shamela.ws/book/1727/4,0,4
1268,[إِلَيْهِ، يَقُومُ مَقَامَ حَدِيثٍ تَامٍّ. فَل...,https://shamela.ws/book/1727/5,0,5
6817,[فهم بِمَا وَصَفْنَا مِنَ الْعِلْمِ وَالسَّتْر...,https://shamela.ws/book/1727/6,0,6
6433,[فَأَمَّا مَا كَانَ مِنْهَا عَنْ قَوْمٍ هُمْ ع...,https://shamela.ws/book/1727/7,0,7
2207,[قَدْ شَرَحْنَا مِنْ مَذْهَبِ الْحَدِيثِ وَأَه...,https://shamela.ws/book/1727/8,0,8
17,وَاعْلَمْ، وَفَّقَكَ اللَّهُ تَعَالَى، أَنَّ ا...,https://shamela.ws/book/1727/9,0,9
5560,[وَالدَّلِيلُ عَلَى أَنَّ الَّذِي قُلْنَا مِنْ...,https://shamela.ws/book/1727/10,0,10


In [ ]:
new_df.to_json("Final_muslim",orient='records',force_ascii = False)